In [24]:
import pandas as pd
import numpy as np

from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [25]:
df = pd.read_csv('../../../data/full_cleaned_data.csv')
df

,Unnamed: 0,year,price,km,range,max_vel,zero_hundred,high_range,performance,four_wheel_drive,...,color_BLUE,color_CHERRY,color_GREY,color_RED,color_SILVER,color_WHITE,country_france,country_italy,country_portugal,country_spain
0,0,2024,99600.0,503.0,634.0,250.0,3.2,False,False,True,...,False,False,False,False,False,True,False,False,False,True
1,1,2024,102270.0,99.0,634.0,250.0,3.2,False,False,True,...,False,False,False,False,False,True,False,False,False,True
2,2,2024,102270.0,0.0,634.0,250.0,3.2,False,False,True,...,False,False,False,False,False,True,False,False,False,True
3,3,2024,90340.0,577.0,634.0,250.0,3.2,False,False,True,...,False,False,False,False,False,False,False,False,False,True
4,4,2024,101050.0,3652.0,600.0,322.0,2.1,False,True,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1827,2058,2022,45500.0,69.0,514.0,250.0,3.7,False,True,True,...,False,False,False,False,False,True,False,True,False,False
1828,2059,2023,42000.0,82.0,533.0,217.0,5.0,True,False,True,...,False,False,False,False,False,False,False,True,False,False
1829,2060,2022,43000.0,92.0,514.0,250.0,3.7,False,True,True,...,False,False,False,False,False,False,False,True,False,False
1830,2061,2022,41000.0,48.0,533.0,217.0,5.0,True,False,True,...,False,False,True,False,False,False,False,True,False,False


In [26]:
df.isna().sum()

Unnamed: 0           0
year                 0
price                0
km                   0
range                0
max_vel              0
zero_hundred         0
high_range           0
performance          0
four_wheel_drive     0
rear_wheel_drive     0
sell_type_Nuevo      0
sell_type_Ocasion    0
car_type_Model 3     0
car_type_Model S     0
car_type_Model X     0
car_type_Model Y     0
color_BLACK          0
color_BLUE           0
color_CHERRY         0
color_GREY           0
color_RED            0
color_SILVER         0
color_WHITE          0
country_france       0
country_italy        0
country_portugal     0
country_spain        0
dtype: int64

In [27]:
df.dtypes

Unnamed: 0             int64
year                   int64
price                float64
km                   float64
range                float64
max_vel              float64
zero_hundred         float64
high_range              bool
performance             bool
four_wheel_drive        bool
rear_wheel_drive        bool
sell_type_Nuevo         bool
sell_type_Ocasion       bool
car_type_Model 3        bool
car_type_Model S        bool
car_type_Model X        bool
car_type_Model Y        bool
color_BLACK             bool
color_BLUE              bool
color_CHERRY            bool
color_GREY              bool
color_RED               bool
color_SILVER            bool
color_WHITE             bool
country_france          bool
country_italy           bool
country_portugal        bool
country_spain           bool
dtype: object

In [28]:
for column in df.columns:
    print(df[column].unique())
    print('=' * 50)

[   0    1    2 ... 2060 2061 2062]
[2024 2023 2018 2019 2022 2020 2021 2025]
[ 99600. 102270.  90340. ...  29200.  37600.  60600.]
[5.030e+02 9.900e+01 0.000e+00 5.770e+02 3.652e+03 1.700e+03 6.198e+03
 9.970e+02 2.100e+01 1.393e+03 2.700e+01 2.000e+01 1.700e+01 2.500e+01
 2.203e+03 4.790e+03 4.400e+02 2.230e+02 3.853e+03 1.726e+03 6.578e+03
 5.580e+03 2.600e+01 3.382e+03 3.906e+03 3.325e+03 1.300e+01 6.018e+03
 3.900e+01 7.566e+03 6.630e+03 1.202e+03 4.446e+03 1.500e+01 2.051e+03
 3.649e+03 5.936e+03 1.015e+03 1.405e+03 3.400e+01 3.295e+03 6.000e+01
 9.200e+01 4.086e+03 5.897e+03 3.229e+03 2.080e+02 3.300e+01 2.300e+01
 1.800e+01 6.314e+03 8.812e+03 8.298e+03 1.317e+03 1.991e+03 4.462e+03
 2.707e+03 7.086e+03 2.036e+03 9.670e+02 5.500e+01 4.010e+02 5.650e+02
 2.291e+03 9.000e+00 7.000e+00 8.864e+03 5.955e+03 5.727e+03 1.525e+03
 1.200e+01 3.960e+02 5.127e+03 2.501e+03 2.857e+03 6.120e+02 2.582e+03
 4.053e+03 5.000e+00 6.890e+02 9.500e+01 5.400e+01 8.310e+02 3.100e+01
 4.371e+03 1.011

In [29]:
df.drop(columns=["Unnamed: 0"], inplace=True)
df.drop_duplicates(inplace=True)

#df = pd.get_dummies(data=df, columns=['sell_type', 'car_type'], drop_first=True)

In [30]:
df.dtypes

year                   int64
price                float64
km                   float64
range                float64
max_vel              float64
zero_hundred         float64
high_range              bool
performance             bool
four_wheel_drive        bool
rear_wheel_drive        bool
sell_type_Nuevo         bool
sell_type_Ocasion       bool
car_type_Model 3        bool
car_type_Model S        bool
car_type_Model X        bool
car_type_Model Y        bool
color_BLACK             bool
color_BLUE              bool
color_CHERRY            bool
color_GREY              bool
color_RED               bool
color_SILVER            bool
color_WHITE             bool
country_france          bool
country_italy           bool
country_portugal        bool
country_spain           bool
dtype: object

In [31]:
df.describe()

,year,price,km,range,max_vel,zero_hundred
count,1832.000000,1832.000000,1832.000000,1832.000000,1832.000000,1832.000000
mean,2023.657751,56856.542031,318.940502,553.077511,229.319869,4.624945
std,1.798174,24925.282716,1207.209696,58.761926,25.204554,1.176042
min,2018.000000,25400.000000,0.000000,409.000000,201.000000,2.100000
25%,2024.000000,42631.250000,0.000000,514.000000,201.000000,3.700000
50%,2024.000000,50347.500000,0.000000,543.000000,217.000000,4.400000
75%,2025.000000,57952.500000,49.000000,600.000000,250.000000,5.200000
max,2025.000000,133390.000000,9771.000000,702.000000,322.000000,6.900000


In [32]:
X = df.drop(columns=['price'])  # Características (sin la columna objetivo)
y = df['price']                # Columna objetivo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1337)

In [33]:
boosters = ['gbtree', 'gblinear', 'dart']
profundidades = [3, 5, 7, 9]
estimators = [25, 50, 75, 100, 150, 200]

mejor_clf = {'profundidad':'',
              'booster':'',
              'estimator': '',
              'mae':'',
              'mse':'',
              'rmse':'',
              'r2':''}
mejor_rmse = 999999999999999

In [34]:
for booster in boosters:
    for estimator in estimators:
        for profundidad in profundidades:
            # Crear el modelo de RandomForest
            xgb_model = XGBRegressor(n_estimators=estimator, booster=booster, max_depth=profundidad, random_state=1337)
            
            # Entrenar el modelo
            xgb_model.fit(X_train, y_train)
            
            # Realizar las predicciones
            y_pred = xgb_model.predict(X_test)
            
            # Calcular las métricas
            mae = mean_absolute_error(y_test, y_pred)
            mse = mean_squared_error(y_test, y_pred)
            rmse = np.sqrt(mse)
            r2 = r2_score(y_test, y_pred)
            
            if rmse <= mejor_rmse:
                mejor_rmse = rmse
                mejor_clf['profundidad'] = profundidad
                mejor_clf['booster'] = booster
                mejor_clf['estimator'] = estimator
                mejor_clf['mae'] = mae
                mejor_clf['mse'] = mse
                mejor_clf['rmse'] = rmse
                mejor_clf['r2'] = r2

c:\Users\Tore\miniconda3\envs\tesla_env\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:41:23] WARNING: D:\bld\xgboost-split_1733179550881\work\src\learner.cc:740: 
Parameters: { "max_depth" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\Tore\miniconda3\envs\tesla_env\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:41:23] WARNING: D:\bld\xgboost-split_1733179550881\work\src\learner.cc:740: 
Parameters: { "max_depth" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\Tore\miniconda3\envs\tesla_env\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:41:23] WARNING: D:\bld\xgboost-split_1733179550881\work\src\learner.cc:740: 
Parameters: { "max_depth" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\Tore\miniconda3\envs\tesla_env\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:41:23] WARNING: D:\bld\xgboost-split_1733179550881\work\src\learner.cc:740: 
Parameters: { "max_depth" } are not used.

  warnings.warn(smsg, UserWarni

In [35]:
mejor_clf

{'profundidad': 5,
 'booster': 'gbtree',
 'estimator': 50,
 'mae': np.float64(1651.75087810797),
 'mse': np.float64(5053526.248596098),
 'rmse': np.float64(2248.004948525714),
 'r2': 0.9911815371924048}

In [36]:
XGBR = XGBRegressor(n_estimators=mejor_clf['estimator'], booster=mejor_clf['booster'], max_depth=mejor_clf['profundidad'], random_state=1337)
XGBR

XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=50, n_jobs=None,
             num_parallel_tree=None, random_state=1337, ...)

In [37]:
XGBR.fit(X_train, y_train)
predicciones = XGBR.predict(X_test)

In [38]:
X_test['predicted_price'] = predicciones
X_test['real_price'] = df.loc[X_test.index, 'price']
X_test

,year,km,range,max_vel,zero_hundred,high_range,performance,four_wheel_drive,rear_wheel_drive,sell_type_Nuevo,...,color_GREY,color_RED,color_SILVER,color_WHITE,country_france,country_italy,country_portugal,country_spain,predicted_price,real_price
536,2020,44.0,530.0,261.0,3.4,True,True,True,False,False,...,False,False,False,True,True,False,False,False,32114.427734,32500.0
969,2025,4642.0,565.0,217.0,5.9,True,False,False,True,True,...,False,False,False,True,False,False,False,True,44335.777344,43970.0
627,2024,815.0,528.0,262.0,3.1,False,True,True,False,True,...,False,False,False,False,False,True,False,False,52500.187500,57285.0
219,2024,0.0,629.0,201.0,4.4,True,False,False,False,True,...,False,True,False,False,False,False,True,False,54679.781250,52715.0
1144,2025,0.0,513.0,201.0,6.1,False,False,False,True,True,...,False,False,True,False,False,False,True,False,43297.261719,44675.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1180,2025,0.0,576.0,250.0,3.9,False,False,True,False,True,...,False,False,False,True,False,False,True,False,114522.218750,110975.0
261,2024,0.0,533.0,217.0,5.0,True,False,False,False,True,...,False,False,False,False,False,False,True,False,53228.640625,56215.0
984,2025,29.0,514.0,250.0,3.7,False,True,True,False,True,...,False,False,False,True,False,False,False,True,54627.949219,50990.0
1810,2020,61.0,560.0,233.0,4.6,True,False,True,False,False,...,True,False,False,False,False,True,False,False,31773.699219,31000.0


In [39]:
X_test.to_csv('../modeling_results_XGBoost.csv')